In [13]:
pip install ee

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
pip install geemap

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import ee
import geemap  
ee.Authenticate()
ee.Initialize(project='ee-kmd232323')

In [29]:

maha = ee.FeatureCollection("users/kmd232323/Maharashtra_shape")
years = [2021, 2022]
months = range(5, 12)


# Function to calculate NDVI
def addNDVI(image):
    ndvi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('NDVI')
    return image.addBands(ndvi)

# Function to calculate NDWI
def addNDWI(image):
    ndwi = image.normalizedDifference(['sur_refl_b04', 'sur_refl_b02']).rename('NDWI')
    return image.addBands(ndwi)

# Function to scale LST to Celsius
def scaleLST(image):
    lst_celsius = image.select('LST_Day_1km').multiply(0.02).subtract(273.15).rename('LST')
    return image.addBands(lst_celsius)

# Function to retrieve images for the 1st and 15th/16th of each month
def get_ndvi_ndwi_for_month(year, month):
    """
    Retrieves a representative image from the MODIS/061/MOD09GA collection for the given year and month,
    and computes NDVI and NDWI on that image.
    """
    # Define the start and end dates for the month
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    
    # Load the MOD09GA collection (Surface Reflectance) and filter by date and region
    mod09ga = ee.ImageCollection("MODIS/061/MOD09GA") \
                .filterBounds(maha) \
                .filterDate(start_date, end_date)
    
    # Map the NDVI and NDWI functions over the collection
    mod09ga = mod09ga.map(addNDVI).map(addNDWI)
    
    # Return the first available image with NDVI and NDWI computed
    return mod09ga.first()


# Function to retrieve LST data for the 1st and 15th/16th of each month
def get_lst_for_month(year, month):
    """
    Retrieves a representative LST image (first available image) 
    from the MODIS/061/MOD11A1 collection for the given year and month.
    The image is scaled to Celsius using the scaleLST function.
    """
    # Define the start and end dates for the month
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    
    # Load the MOD11A1 collection (Land Surface Temperature)
    mod11a1 = ee.ImageCollection("MODIS/061/MOD11A1") \
                .filterBounds(maha) \
                .filterDate(start_date, end_date) \
                .map(scaleLST)
    
    # Return the first available image in the month
    return mod11a1.first()

# Function to retrieve CHIRPS rainfall data for the 1st and 15th/16th of each month
def get_rainfall_for_month(year, month):
    """
    Retrieves a representative monthly rainfall image from the CHIRPS daily collection 
    for the given year and month. The image is computed as the sum of daily rainfall values.
    """
    # Define the start and end dates for the month
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    
    # Load the CHIRPS daily rainfall data and filter by date and region
    chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
                .filterBounds(maha) \
                .filterDate(start_date, end_date)
    
    # Compute the monthly sum of rainfall
    monthly_rainfall = chirps.sum().rename("Rainfall")
    
    # Set the time property to the start of the month (for consistency)
    monthly_rainfall = monthly_rainfall.set('system:time_start', start_date.millis())
    
    return monthly_rainfall


ndvi_ndwi_images = []
for year in years:
    for month in months:
        ndvi_ndwi_image = get_ndvi_ndwi_for_month(2021, 5)
        if ndvi_ndwi_image:
            ndvi_ndwi_images.append(ndvi_ndwi_image)

lst_images = []
for year in [2021, 2022]:
    for month in range(5, 12):  # May to November
        lst_img = get_lst_for_month(year, month)
        if lst_img:
            lst_images.append(lst_img)

rainfall_images = []
for year in [2021, 2022]:
    for month in range(5, 12):  # May to November
        img = get_rainfall_for_month(year, month)
        if img:
            rainfall_images.append(img)

# Convert the list to an ImageCollection
rainfall_collection = ee.ImageCollection(rainfall_images)
print("Number of monthly rainfall images in collection:", rainfall_collection.size().getInfo())

# Convert the list of LST images to an ImageCollection
lst_collection = ee.ImageCollection(lst_images)
print("Number of LST images in collection:", lst_collection.size().getInfo())

ndvi_ndwi_collection = ee.ImageCollection(ndvi_ndwi_images)
print("Number of images in monthly_collection:", ndvi_ndwi_collection.size().getInfo())


# Remove any null entries (dates with no available images)
ndvi_ndwi_images = [img for img in ndvi_ndwi_images if img]
lst_images = [img for img in lst_images if img]
rainfall_images = [img for img in rainfall_images if img]

# Convert lists to ImageCollections
ndvi_ndwi_collection = ee.ImageCollection(ndvi_ndwi_images).map(addNDVI).map(addNDWI)
lst_collection = ee.ImageCollection(lst_images)
rainfall_collection = ee.ImageCollection(rainfall_images)

# Function to merge all bands from different collections into a single image
def merge_bands(lst_img):
    # Get the date of the LST image
    date = ee.Date(lst_img.get('system:time_start'))

    # Find the closest NDVI/NDWI image by sorting by date difference
    ndvi_ndwi_img = ndvi_ndwi_collection.sort('system:time_start').map(
        lambda img: img.set('diff', ee.Number(img.date().difference(date, 'day')).abs())
    ).sort('diff').first()

    # Same for rainfall
    rainfall_img = rainfall_collection.sort('system:time_start').map(
        lambda img: img.set('diff', ee.Number(img.date().difference(date, 'day')).abs())
    ).sort('diff').first()

    # Start with the LST image
    merged = lst_img

    # Add NDVI/NDWI if available
    merged = ee.Algorithms.If(ndvi_ndwi_img,
        merged.addBands(ee.Image(ndvi_ndwi_img).select(['NDVI', 'NDWI'])),
        merged)

    # Add rainfall if available
    merged = ee.Algorithms.If(rainfall_img,
        ee.Image(merged).addBands(ee.Image(rainfall_img).select('Rainfall')),
        merged)

    return ee.Image(merged)



merged_collection = lst_collection.map(merge_bands)
print("Number of merged images:", merged_collection.size().getInfo())

Number of monthly rainfall images in collection: 14
Number of LST images in collection: 14
Number of images in monthly_collection: 14
Number of merged images: 14


In [36]:

from IPython.display import Image, display

# Assume Earth Engine is already initialized and merged_collection is defined.
# Also assume maha is your Maharashtra FeatureCollection.

# Define visualization parameters for NDVI
vis_params = {
    'bands': ['NDVI'],
    'min': 0,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

# Get the region geometry as a GeoJSON-like dictionary (used for thumbnail export)
region = maha.geometry().getInfo()  # This returns the full geometry dict


# Convert the merged_collection to a list to iterate over images
num_images = merged_collection.size().getInfo()
print("Total images in merged_collection:", num_images)

merged_list = merged_collection.toList(num_images)

# Loop over each image, get its thumbnail URL, and display it
for i in range(num_images):
    img = ee.Image(merged_list.get(i))
    # Format the date for display (e.g., "2021-05")
    date_str = ee.Date(img.get('system:time_start')).format('YYYY-MM').getInfo()
    
    # Prepare thumbnail parameters. Dimensions can be adjusted as needed.
    thumb_params = {
        'min': vis_params['min'],
        'max': vis_params['max'],
        'palette': vis_params['palette'],
        'region': region,
        'dimensions': 512  # Adjust the dimensions if necessary
    }
    
    # Get the thumbnail URL for the NDVI band (you can change vis_params['bands'] if desired)
    thumb_url = img.select(vis_params['bands']).getThumbURL(thumb_params)
    
    # Print the date and display the image
    print("Image for:", date_str)
    display(Image(url=thumb_url))


Total images in merged_collection: 14
Image for: 2021-05


Image for: 2021-06


Image for: 2021-07


Image for: 2021-08


Image for: 2021-09


...

Image for: 2022-07


Image for: 2022-08


Image for: 2022-09


Image for: 2022-10


Image for: 2022-11
